In [53]:
import pandas as pd
data = []
url = []
name = []
stock = []
ver = []
web = []
pics = []

In [54]:
casn = [ 
    "103-63-9",    # (2-Bromoethyl) Benzene (a known fentanyl precursor)
    "1609-66-1",   # Norfentanyl (NSC 89293)
    "288573-56-8", #
    "62811-39-6",   # (2-nitroprop-1-en-1-yl)benzene (1-phenyl-2-nitropropene; P2NP)
    "98-86-2"]

In [55]:
#removing duplicate numbers from the list
casn = list(dict.fromkeys(casn))
print(len(casn))

5


In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time


for i in range (len(casn)):
    for pg in range(1, 3):
        try:
            # URL of the webpage you want to scrape
            url1 = 'https://www.chemicalbook.com/ProductList_En.aspx?kwd='
            CAS = casn[i]
            url2 = '&a=China&left=True&page='
            url3 = '#J_Condition'
            u = url1 + CAS + url2 + str(pg) + url3
            
            # Initialize the Chrome driver
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service)
            
            # Open the webpage
            driver.get(u)

            # Extracting the elements information
            Box_elements = driver.find_elements(By.CLASS_NAME, 'ProLBox')

            # Initialize the ActionChains
            actions = ActionChains(driver)

            for Box in Box_elements:
                data.append(Box.text)
                names = Box.find_elements(By.CLASS_NAME, 'list-title')
                sec_element = Box.find_elements(By.CLASS_NAME, 'icon_prolist.SecurityPro')
                pic = Box.find_elements(By.CLASS_NAME, 'TouchArr2')

                if len(names) == 0:
                    continue
                else:
                    name.append(names[0].text)
                    web.append(names[0].get_attribute('href'))

                if len(sec_element) == 0:
                    ver.append('')
                else :
                    actions.move_to_element(sec_element[0]).perform()
                    ver.append(sec_element[0].text)

                if len(pic) == 0:
                    pics.append('')
                else :
                    pics.append(names[0].get_attribute('img'))
                    print (names[1].get_attribute('img').text)
            # Don't forget to close the browser!
            driver.quit()

            if len(Box_elements) < 20:
                print(str(round((i/len(casn))*100, 2)) + '% completed')
                break

        except Exception as e:
            print('Error \t',e, '\n at: ' + str(u))
            driver.quit()

        #progress bar as percentage of completion of the scraping
        print(str(round((i/len(casn))*100, 2)) + '% completed')

        #set sleep time to avoid being blocked
        time.sleep(0.5)

Error 	 list index out of range 
 at: https://www.chemicalbook.com/ProductList_En.aspx?kwd=103-63-9&a=China&left=True&page=1#J_Condition
0.0% completed
Error 	 list index out of range 
 at: https://www.chemicalbook.com/ProductList_En.aspx?kwd=103-63-9&a=China&left=True&page=2#J_Condition
0.0% completed
Error 	 list index out of range 
 at: https://www.chemicalbook.com/ProductList_En.aspx?kwd=1609-66-1&a=China&left=True&page=1#J_Condition


In [ ]:
print(pics)

[None, '', None, '', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '', '', '', None, '', '', '', None, '', '', '', '', '', '', '', '', None, '', '', '', None, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', None, '', None, None, '', None, None, None, '', '', '', None, '', '', '', None, None, '', None, None, None, None, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', None, None, None, None, '', '', '', '', '', '', None, '', '', '', '', '', '', '', None, '', '', '', '', '', '', '', '', '', '', '', None, '', '', '', '', '', '']


In [ ]:
id = []
n = []
cas = []
pn = []
verified = []

In [ ]:
for v in ver:
    if v != '':
        verified.append(v.split('\n')[2].split(':')[1])
    else:
        verified.append('')

In [ ]:
for i in range(len(data)):
    id.append('ChemBook_'+str(i))
    for na in name:
        if na in data[i]:
            n.append(na)
            break

    if 'CAS:' in data[i]:
        cas.append(str(data[i].split('CAS:')[1].split('\n')[0]))
    elif 'Cas:' in data[i]:
        cas.append(str(data[i].split('Cas:')[1].split('\n')[0]))
    elif 'cas:' in data[i]:
        cas.append(str(data[i].split('cas:')[1].split('\n')[0]))
    else:
        cas.append('')


    if 'Product Name:' in data[i]:
        pn.append(str(data[i].split('Product Name:')[1].split('\n')[0]))
    else:
        pn.append('')


In [ ]:
print (len(id), len(n), len(verified), len(cas), len(pn), len(web))

141 141 141 141 141 141


In [ ]:
data_df = pd.DataFrame(list(zip(id,n,cas,pn,verified,web)), columns =['ID','Name','CAS','Product_Name','Verified_On','Website_URL'])
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            141 non-null    object
 1   Name          141 non-null    object
 2   CAS           141 non-null    object
 3   Product_Name  141 non-null    object
 4   Verified_On   141 non-null    object
 5   Website_URL   141 non-null    object
dtypes: object(6)
memory usage: 6.7+ KB


In [ ]:
data_df.to_csv('ChemBook_final_extra.csv', index = False)
data_df.head(50)

,ID,Name,CAS,Product_Name,Verified_On,Website_URL
0,ChemBook_0,"Shanghai Longyu Biotechnology Co., Ltd.",103-63-9,(2-Bromoethyl)benzene,2019-09-05,https://www.chemicalbook.com/manufacturer/shan...
1,ChemBook_1,Siyang Liaoning Biochemical Products Co. Ltd,103-63-9,(2-Bromoethyl)benzene,2021-04-24,https://www.chemicalbook.com/ShowSupplierProdu...
2,ChemBook_2,"LY Global chemicals co.,ltd .",103-63-9,(2-Bromoethyl)benzene,2023-03-06,https://www.chemicalbook.com/ShowSupplierProdu...
3,ChemBook_3,"Wuhan Boyuan Import & Export Co., LTD",103-63-9,(2-Bromoethyl)benzene / 2-Phenylethyl bromide,,http://www.whby-chem.com/
4,ChemBook_4,"Jinan Finer Chemical Co., Ltd",103-63-9,(2-Bromoethyl)benzene,2019-10-17,http://www.finerchem.com/
5,ChemBook_5,"Shanghai Yunao International Trade Co., Ltd",103-63-9,(2-Bromoethyl)benzene,2023-04-21,https://www.chemicalbook.com/manufacturer/shan...
6,ChemBook_6,"Hebei Guanlang Biotechnology Co,.LTD",103-63-9,(2-Bromoethyl) Benzene,2023-12-12,https://www.chemicalbook.com/manufacturer/hebe...
7,ChemBook_7,"Wuhan wingroup Pharmaceutical Co., Ltd",103-63-9,(2-Bromoethyl)benzene,2021-06-24,http://www.whwingroup.com/
8,ChemBook_8,Hebei Dangtong Import and export Co LTD,103-63-9,(2-Bromoethyl)benzene,,https://www.chemicalbook.com/ShowSupplierProdu...
9,ChemBook_9,"Anhui Yiao New Material Technology Co., Ltd",103-63-9,(2-Bromoethyl)benzene,2023-06-29,https://www.chemicalbook.com/manufacturer/Yiao...


In [ ]:
data_df.tail(50)

,ID,Name,CAS,Product_Name,Verified_On,Website_URL
91,ChemBook_91,"Beijing Haite lesk Technology Co., Ltd",288573-56-8,1-BOC-4-(4-FLUORO-PHENYLAMINO)-PIPERIDINE,,https://www.chemicalbook.com/ShowSupplierProdu...
92,ChemBook_92,"Hubei Enxing Biotechnology Co., Ltd",288573-56-8,1-BOC-4-(4-FLUORO-PHENYLAMINO)-PIPERIDINE,,http://www.exbio-tech.com/cn/
93,ChemBook_93,"Hubei xingongli Chemical Co., Ltd",288573-56-8,ert-butyl4-(4-fluoroanilino)piperidine-1-carb...,,http://www.xinhonglichem.com/
94,ChemBook_94,"Hubei Tuobang Chemical Co., Ltd.",288573-56-8,ert-butyl4-(4-fluoroanilino)piperidine-1-carb...,,https://www.chemicalbook.com/ShowSupplierProdu...
95,ChemBook_95,"Suzhou Jiayinuo Technology Co., Ltd.",288573-56-8,tert-butyl 4-(4-fluoroanilino)piperidine-1-ca...,,https://www.chemicalbook.com/ShowSupplierProdu...
96,ChemBook_96,"Hubei DAHAO Chemical Co., Ltd.",288573-56-8,ert-butyl4-(4-fluoroanilino)piperidine-1-carb...,,https://www.chemicalbook.com/ShowSupplierProdu...
97,ChemBook_97,"Shenyang Maud Pharmaceutical Technology Co., Ltd.",288573-56-8,1-BOC-4-(4-FLUORO-PHENYLAMINO)-PIPERIDINE,,http://www.modechem.com/
98,ChemBook_98,"Suzhou Haihe Pharmaceutical Technology Co., Ltd.",288573-56-8,1-BOC-4-(4-FLUORO-PHENYLAMINO)-PIPERIDINE,,http://www.haihepharm.cn/
99,ChemBook_99,"Tangyuan Life Science (Shanghai) Co., Ltd",288573-56-8,1-BOC-4-(4-FLUORO-PHENYLAMINO)-PIPERIDINE,,http://www.townyuan.com/
100,ChemBook_100,"Hebei Paixing Technology Co., LTD",288573-56-8,tert-butyl 4-(4-fluoroanilino)piperidine-1-ca...,,https://www.chemicalbook.com/ShowSupplierProdu...
